In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.7 MB/s eta 0:00:00


In [1]:
import os
from datasets import Dataset
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, losses, evaluation, InputExample
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
# Установка параметров ===
model_name = "distilroberta-base"
custom_model_path = "./custom_transformer"
batch_size = 8
num_epochs = 8
evaluation_steps = 100

# Загружаем конфигурацию и изменяем dropout
config = AutoConfig.from_pretrained(model_name)
config.hidden_dropout_prob = 0.2  # Dropout в скрытых слоях
config.attention_probs_dropout_prob = 0.2  # Dropout в механизме внимания

# Загружаем предобученную модель с модифицированной конфигурацией
transformer_model = AutoModel.from_pretrained(model_name, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Убеждаемся, что папка для модели существует
os.makedirs(custom_model_path, exist_ok=True)

# Сохраняем кастомную модель и токенизатор
transformer_model.save_pretrained(custom_model_path)
tokenizer.save_pretrained(custom_model_path)

# === Загрузка кастомной модели в sentence-transformers ===
word_embedding_model = models.Transformer(custom_model_path, max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode="mean")
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

print("Модель успешно загружена с кастомной конфигурацией")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Модель успешно загружена с кастомной конфигурацией


In [3]:
# === Загрузка данных триплетов ===
triplets_df = pd.read_pickle("/content/data/house_triplets.pkl")

# === Преобразование данных в InputExample ===
examples = [
    InputExample(texts=[row["anchor"], row["response"], row["neg_response"]])
    for _, row in triplets_df.iterrows()
]

# === Вывод статистики ===
print(f"Общее количество триплетов: {len(triplets_df)}\n")

# === Вывод примеров триплетов ===
print("Примеры триплетов:\n")
for i in range(5):
    ex = examples[i].texts
    print(f"Анкер: {ex[0]}\nПозитивный ответ: {ex[1]}\nНегативный ответ: {ex[2]}\n")

Общее количество триплетов: 7417

Примеры триплетов:

Анкер: Fair enough. I dont like healthy Patients. The 29 year old female!
Позитивный ответ: The one who cant talk, I liked that part.
Негативный ответ: All right. Let me give you a little girlfriend 101. Usually the first move out of the gate is you withhold sex, but that will work better after Sheldon hits

Анкер: Shouldnt we be speaking to the Patient before we start diagnosing?
Позитивный ответ: Is she a doctor
Негативный ответ: Yeah, but we havent heard a thing from the adoption agency and it has been weeks!

Анкер: Isnt treating Patients why we became doctors?
Позитивный ответ: No, treating illnesses is why we became doctors, treating Patients is what makes most doctors miserable.
Негативный ответ: Yeah, well, its in our contract to serve on a university committee. And frankly, this is one I believe in. Okay, here we go. Encouraging more women to pursue a

Анкер: Wernickies encephalopathy?
Позитивный ответ: No, blood thiamine l

In [4]:
# === Разделение данных на train и validation ===
train_examples, valid_examples = train_test_split(examples, test_size=0.2, random_state=42)

# === Вывод статистики после разбиения ===
print(f"Данные разделены на train и validation:")
print(f"Train: {len(train_examples)} триплетов ({100 * len(train_examples) / len(examples):.1f}%)")
print(f"Validation: {len(valid_examples)} триплетов ({100 * len(valid_examples) / len(examples):.1f}%)\n")

# === Вывод примеров из train и validation ===
print("Пример из train:")
train_ex = train_examples[0].texts
print(f"Анкер: {train_ex[0]}\nПозитивный ответ: {train_ex[1]}\nНегативный ответ: {train_ex[2]}\n")

print("Пример из validation:")
valid_ex = valid_examples[0].texts
print(f"Анкер: {valid_ex[0]}\nПозитивный ответ: {valid_ex[1]}\nНегативный ответ: {valid_ex[2]}\n")

Данные разделены на train и validation:
Train: 5933 триплетов (80.0%)
Validation: 1484 триплетов (20.0%)

Пример из train:
Анкер: !MPPP.
Позитивный ответ: Either we play 20,000 questions with Every street drug, or instead of talking to him, we talk to the lesion.
Негативный ответ: Ill take that down right now.

Пример из validation:
Анкер: Dextromethorphan. As in cough syrup?
Позитивный ответ: He wasnt taking it for his cough. Its cheap, available at your corner drug store, and taken in high doses, its a dumb drug.
Негативный ответ: If youd have let me bring the chloroform, we wouldnt have had to put up with all this jibberjabber.



In [5]:
# === Создание DataLoader ===
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)
valid_dataloader = DataLoader(valid_examples, shuffle=False, batch_size=batch_size)

# === Задание функции потерь ===
train_loss = losses.TripletLoss(model=model)

In [6]:
# === Оценка модели перед обучением ===
print("Оценка модели перед обучением...")
dev_evaluator = evaluation.TripletEvaluator.from_input_examples(valid_examples, name="dev_evaluator", batch_size=batch_size)
pretrain_score = model.evaluate(dev_evaluator)
print(f"Точность перед обучением: {pretrain_score['dev_evaluator_cosine_accuracy']:.4f}")

Оценка модели перед обучением...
Точность перед обучением: 0.5451


In [8]:
contains_response = triplets_df.apply(lambda row: row["response"] in row["anchor"], axis=1)
percentage_contains = contains_response.mean() * 100

print(f"Процент анкорных реплик, которые содержат ответ полностью: {percentage_contains:.2f}%")


Процент анкорных реплик, которые содержат ответ полностью: 0.03%


In [13]:
# === Запуск обучения (без вычисления точности в процессе) ===
print("Начинается обучение...")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=int(len(train_dataloader) * num_epochs * 0.05),  # 10% от общего числа шагов
    optimizer_params={'lr': 2e-5},  # Меньше learning rate
    output_path="./sbert_test"
)

# === Финальная оценка модели ===
print("Оценка модели после обучения...")
val_evaluator = evaluation.TripletEvaluator.from_input_examples(valid_examples, name="final_evaluator", batch_size=batch_size)
final_score = model.evaluate(val_evaluator)

print(f"Финальная точность на валидации: {final_score['final_evaluator_cosine_accuracy']:.4f}")

Начинается обучение...


Step,Training Loss
500,0.369600
1000,0.436200
1500,0.390800
2000,0.261600
2500,0.210500
3000,0.187700
3500,0.140600
4000,0.114100
4500,0.113600
5000,0.070800


Оценка модели после обучения...
Финальная точность на валидации: 0.8848


In [14]:
# === Сохранение модели ===
model.save("models")
print("Модель сохранена локально в папку models")

Модель сохранена локально в папку models


In [28]:
import shutil
from google.colab import files

# Упаковка модели в архив ZIP
shutil.make_archive("sbert_model", "zip", "./sbert_test")

# Скачивание архива
files.download("sbert_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
# Указываем путь для сохранения модели
model_path = "./models/chatbot_sentence-transformer"

# Сохраняем модель и токенизатор
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

print(f"Модель и токенизатор сохранены в директории {model_path}")

Модель и токенизатор сохранены в директории ./models/chatbot_sentence-transformer


In [34]:
from sentence_transformers import SentenceTransformer

# Загрузка модели из папки
model = SentenceTransformer("models/chatbot_sentence-transformer")

In [39]:
from huggingface_hub import notebook_login
from sentence_transformers import SentenceTransformer

# Вход в Hugging Face
notebook_login()  # Введи свой токен Hugging Face при запросе

# Загружаем модель из сохранённой папки
model_path = "models/chatbot_sentence-transformer"
model = SentenceTransformer(model_path)

# Перезаписываем модель в существующем репозитории с новым именем
model.push_to_hub("nikatonika/chatbot_biencoder", commit_message="Update model after training")

# Если токенизатор был использован отдельно, можно загрузить его тоже
# tokenizer.push_to_hub("nikatonika/chatbot_biencoder", commit_message="Update tokenizer after training")

print("✅ Модель успешно перезаписана в репозитории chatbot_biencoder на Hugging Face!")


model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

✅ Модель успешно перезаписана в репозитории chatbot_biencoder на Hugging Face!
